# 8. Совместное использование C++ и Python. Модуль CherryPy для создания веб-приложений.

## 8.1. Совместное использование с C++
### 8.1.1. swig

In [84]:
%%writefile swtest.c

int fact(int n) {
    if (n <= 1) return 1;
    else return n * fact(n-1);
}

Overwriting swtest.c


In [83]:
%%writefile swtest.swig

%module swtest
%{
    extern int fact(int n);
%}

extern int fact(int n);

Overwriting swtest.swig


In [86]:
%%bash
swig -python swtest.swig
gcc -fPIC -c swtest.c swtest_wrap.c -I/usr/include/python3.4
gcc -shared swtest.o swtest_wrap.o -o _swtest.so

In [20]:
import swtest
print(swtest.fact(10))

3628800


### 8.1.2. cffi

In [87]:
%%writefile cftest.cpp
#include <omp.h>

extern "C" {
    
struct DataStruct {
  unsigned char *Data;
  int Nx, Ny, Nz;
};

void MarkBoundaries(DataStruct* data) {
    int Nx = data->Nx;
    int Ny = data->Ny;
    int Nz = data->Nz;
    unsigned char * const raw = data->Data;

    #pragma omp parallel for
    for (int x = 1; x < Nx - 1; ++x) {
        for (int y = 1; y < Ny - 1; ++y) {
            for (int z = 1; z < Nz - 1; ++z) {
                if (raw[x + y * Nx + z * Nx * Ny] == 0) {
                    for (int xx = x - 1; xx < x + 2; ++xx) {
                        for (int yy = y - 1; yy < y + 2; ++yy) {
                            for (int zz = z - 1; zz < z + 2; ++zz) {
                                if (raw[xx + yy * Nx + zz * Nx * Ny] == 1) {
                                    raw[x + y * Nx + z * Nx * Ny] = 3;
                                }
                            }
                        }
                    }
                }
            }
        }
    }
}

}

Writing cftest.cpp


In [2]:
%%bash
g++ -fPIC -c cftest.cpp -fopenmp -O3
g++ -fopenmp -shared cftest.o -o _cftest.so

In [3]:
%%writefile cftest.py

from cffi import FFI
import os
_ffi = FFI()
_lib = _ffi.dlopen(os.path.join(os.path.dirname(os.path.realpath(__file__)), "_cftest.so"))
_ffi.cdef("""
typedef struct {
    uint8_t *Data;
    int Nx, Ny, Nz;
} DataStruct;
void MarkBoundaries(struct DataStruct* data);
""")

def MarkBoundaries(raw, size):
    data = _ffi.new("DataStruct[]", 1)
    data[0].Data = _ffi.cast("uint8_t *", raw.ctypes.data)
    data[0].Nx, data[0].Ny, data[0].Nz = size
    dataPtr = _ffi.cast("struct DataStruct *", data)
    
    _lib.MarkBoundaries(dataPtr)

Overwriting cftest.py


In [4]:
import cftest
import numpy

raw = numpy.fromfile("S1_300x300x300.raw", dtype=numpy.uint8)
size = 300, 300, 300

print(raw[131450:131590])

cftest.MarkBoundaries(raw, size)

print(raw[131450:131590])

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 3 3 0 0 0 3 3 3 3 3 3 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 3 3 3 3 3]


### 8.1.3. ctypes

In [9]:
%%writefile ctytest.cpp

extern "C" {
    
void sum(float *a, float *b, float *c, int size)
{
    for (int i = 0; i < size; ++i) {
        c[i] = a[i] + b[i];
    }
}
    
}

Writing ctytest.cpp


In [10]:
%%bash
g++ -fPIC -shared -o _ctytest.so ctytest.cpp

In [88]:
import ctypes
import numpy as np

def build_ctytest_sum():
    dll = ctypes.CDLL('./_ctytest.so', mode=ctypes.RTLD_GLOBAL)
    func = dll.sum
    func.argtypes = [ctypes.POINTER(ctypes.c_float),
                     ctypes.POINTER(ctypes.c_float),
                     ctypes.POINTER(ctypes.c_float),
                     ctypes.c_size_t
                    ]
    return func

def ctytest_sum(a, b, c, size):
    ctytest_sum_impl = build_ctytest_sum()
    a_p = a.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
    b_p = b.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
    c_p = c.ctypes.data_as(ctypes.POINTER(ctypes.c_float))

    ctytest_sum_impl(a_p, b_p, c_p, size)

size = int(1024*1024)

a = np.ones(size).astype('float32')
b = np.ones(size).astype('float32')
c = np.zeros(size).astype('float32')

ctytest_sum(a, b, c, size)

print(c[10])

2.0


## 8.2. Python для создания веб-приложений

Фреймворки:
1. Flask
2. Django
3. Tornado
4. CherryPy

Шаблонные движки:
1. Jinja2
2. Mako

### 8.2.1. Шаблонный движок jinja2

Установка:

    conda install jinja2
    pip install jinja2

Документация: http://jinja.pocoo.org/docs/dev/

In [30]:
%%writefile base.html

<!DOCTYPE html>
<html>
  <head>
    <meta charset="utf-8">
      <title> {{ title }} </title>
  </head>
  <body>
    {% block body %}Некоторый текст{% endblock %}
  </body>
</html>

Overwriting base.html


In [90]:
import jinja2

j2env = jinja2.Environment(loader=jinja2.FileSystemLoader('.'), trim_blocks=True)
print(j2env.get_template('base.html').render(title='Имя блога'))


<!DOCTYPE html>
<html>
  <head>
    <meta charset="utf-8">
      <title> Имя блога </title>
  </head>
  <body>
    Некоторый текст  </body>
</html>


In [91]:
%%writefile body.html
{% extends "base.html" %}

{% block body %}
    {{ super() }}
    <ol>
    {% for item in items %}
      <li>{{ item }}</li>
    {% endfor %}
    </ol>
{% endblock %}

Overwriting body.html


In [92]:
print(j2env.get_template('body.html').render(title='Имя блога 3', items=['яблоко', 'персик', 'ананас']))


<!DOCTYPE html>
<html>
  <head>
    <meta charset="utf-8">
      <title> Имя блога 3 </title>
  </head>
  <body>
        Некоторый текст
    <ol>
          <li>яблоко</li>
          <li>персик</li>
          <li>ананас</li>
        </ol>
  </body>
</html>


### 8.2.2. Модуль CherryPy

Установка:
    
    conda install cherrypy
    pip install cherrypy
    
Документация: http://docs.cherrypy.org/en/latest/index.html

In [59]:
%%writefile app.py
#!/usr/bin/env python
# coding: utf-8
import cherrypy
import jinja2
import os

class Application():
    def __init__(self, blog_name):
        self.blog_name = blog_name
        current_dir = os.path.dirname(os.path.abspath(__file__))
        self.j2env = jinja2.Environment(loader=jinja2.FileSystemLoader(current_dir), trim_blocks=True)
    
    @cherrypy.expose
    def index(self):
        return self.j2env.get_template('base.html').render(title=self.blog_name)
    
    @cherrypy.expose
    def make_list(self, items):
        encoded_items = (x.encode('ISO-8859-1').decode('utf-8') for x in items.split(','))
        return self.j2env.get_template('body.html').render(title=self.blog_name, items=encoded_items)

if __name__ == '__main__':
   cherrypy.quickstart(Application('Тестовый блог'), '/')


Overwriting app.py


При загрузке урла http://127.0.0.1:8080/make_list/яблоко,дорога,кран

Выдаст страницу:

Некоторый текст

1. яблоко
2. дорога
3. кран

### 8.2.3. Модуль CherryPy с Ajax, пример из официальной документации

http://docs.cherrypy.org/en/latest/tutorials.html#tutorial-8-make-it-smoother-with-ajax

In [81]:
%%writefile index.html
<!DOCTYPE html>
<html>
   <head>
     <script src="http://code.jquery.com/jquery-2.0.3.min.js"></script>
     <script type="text/javascript">
       $(document).ready(function() {

         $("#generate-string").click(function(e) {
           $.post("/generator", {"length": $("input[name='length']").val()})
            .done(function(string) {
               $("#the-string").show();
               $("#the-string input").val(string);
            });
           e.preventDefault();
         });

         $("#replace-string").click(function(e) {
           $.ajax({
              type: "PUT",
              url: "/generator",
              data: {"another_string": $("#the-string input").val()}
           })
           e.preventDefault();
         });

         $("#delete-string").click(function(e) {
           $.ajax({
              type: "DELETE",
              url: "/generator"
           })
           .done(function() {
              $("#the-string").hide();
           });
           e.preventDefault();
         });
                
         $.ajax({
           type: "GET",
           url: "/generator"
         }).done(function(string) {
            $("#the-string input").val(string);
        }
         );
       });
     </script>
   </head>
   <body>
     <input type="text" value="20" name="length" />
     <button id="generate-string">Generate string</button>
     <div id="the-string">
         <input type="text" />
         <button id="replace-string">Replace</button>
         <button id="delete-string">Delete it</button>
     </div>
   </body>
</html>

Overwriting index.html


In [79]:
%%writefile tut8.py
import os, os.path
import random
import string

import cherrypy

class StringGenerator(object):
   @cherrypy.expose
   def index(self):
       return open('index.html')

class StringGeneratorWebService(object):
    exposed = True

    @cherrypy.tools.accept(media='text/plain')
    def GET(self):
        return cherrypy.session['mystring']

    def POST(self, length=8):
        some_string = ''.join(random.sample(string.hexdigits, int(length)))
        cherrypy.session['mystring'] = some_string
        return some_string

    def PUT(self, another_string):
        cherrypy.session['mystring'] = another_string

    def DELETE(self):
        cherrypy.session.pop('mystring', None)

if __name__ == '__main__':
    conf = {
        '/': {
            'tools.sessions.on': True,
            'tools.staticdir.root': os.path.abspath(os.getcwd())
        },
        '/generator': {
            'request.dispatch': cherrypy.dispatch.MethodDispatcher(),
            'tools.response_headers.on': True,
            'tools.response_headers.headers': [('Content-Type', 'text/plain')],
        },
    }
    webapp = StringGenerator()
    webapp.generator = StringGeneratorWebService()
    cherrypy.quickstart(webapp, '/', conf)

Overwriting tut8.py
